In [1]:
from Agent import *

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Hyperparameters
gamma = 0.75
nb_trials = 100
nb_episodes = 10000
learning_rate = 7e-6
eps = 1e-7
beta_v = 0.05
beta_e = 0.05

# Prepare

In [3]:
rail_generator = sparse_rail_generator(max_num_cities=10)

env = RailEnv(
    width=30, 
    height=30, 
    number_of_agents=1,
    rail_generator=rail_generator,
    line_generator=sparse_line_generator(),
    obs_builder_object=GlobalObsForRailEnv()
    )

env.stop_penalty = -1
env.invalid_action_penalty = -1
env.alpha = 1

In [4]:
# Save Paths
path = "train_LSTMCell_A2C/" + datetime.now().strftime("%m%d-%H:%M:%S") + "_adam_lr7e-6_g0.75"
log_dir = path+'/logs/'
ckpt_dir = path+'/ckpt/'
train_summary_writer = tf.summary.create_file_writer(log_dir)

2023-01-15 22:29:47.992553: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
agent_id = 0
agent = LSTMAgent(learning_rate, gamma, beta_v, beta_e,  #loss func
                 env, nb_trials, nb_episodes,path,
                 nb_hidden = 128)

# Train

In [6]:
agent.train()

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 3/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 2/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 1/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:262: UserWarning: [WARNING] Changing to Grid mode to place at least 2 cities.
  warnings.warn("[WARNING] Changing to Grid mode to place at least 2 cities.")


In [7]:
agent.model.save(path+'/model.h5')

# Test

In [8]:
#test_model = keras.models.load_model("train_LSTMCell_A2C/0115-19:25:47_adam/model.h5")
test_model = agent.model

In [13]:

def run_episode(env, model):
    next_obs, _ = env.reset()
    score = 0
    reward = 0.0
    action_onehot = np.zeros((agent.nb_actions))
    cell_state = [tf.zeros((1,agent.nb_hidden)),tf.zeros((1,agent.nb_hidden))]
        
    for timestep in range(100):
        input = agent.prepare_input(next_obs, agent_id, action_onehot,reward,timestep)
        
        action_probs, critic_value, cell_state = model([input,cell_state[0],cell_state[1]])
        
        action_probs = tf.squeeze(action_probs)
        action = np.random.choice(agent.nb_actions, p=action_probs.numpy())
        action_onehot = np.zeros((agent.nb_actions))
        action_onehot[action] = 1

        next_obs, all_rewards, dones, info = env.step({agent_id: action})

        for agent_handle in env.get_agent_handles():
            score += all_rewards[agent_handle]

        #render_env(env)
        #print('Timestep {}, action = {}, total score = {}'.format(timestep, action_map[action], score))
        #tf.print(action_probs)

        if dones['__all__']:
            print('All done!')
            print(info['state'][0])
            print(timestep, env._max_episode_steps)
            print(env.agents[agent_id].position, env.agents[agent_id].target)

            if info['state'][0] == TrainState.DONE:
                #render_env(env)
                print('Timestep {}, action = {}, total score = {}'.format(timestep, action_map[action], score))
            return score

    print("Episode didn't finish after 100 timesteps.")
    return score

In [14]:
print(env.agents[0].earliest_departure)
print(env.agents[0].latest_arrival)

2
79


In [15]:
out = []
for i in range(100):
    out.append(run_episode(env, test_model))
    
print(out)

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 3/4
  warnings.warn(city_warning)


All done!
TrainState.MOVING
80 81
(5, 10) (10, 7)
All done!
TrainState.MOVING
97 98
(0, 14) (5, 24)
All done!
TrainState.MOVING
25 26
(22, 23) (21, 24)
All done!
TrainState.DONE
56 80
None (22, 9)
Timestep 56, action = foward, total score = 0
All done!
TrainState.MOVING
50 51
(11, 5) (17, 12)
All done!
TrainState.MOVING
35 36
(9, 0) (8, 5)
All done!
TrainState.DONE
62 73
None (5, 9)
Timestep 62, action = foward, total score = 0
All done!
TrainState.MOVING
55 56
(11, 22) (15, 11)
All done!
TrainState.MOVING
29 30
(20, 4) (22, 9)
All done!
TrainState.MOVING
43 44
(5, 15) (6, 22)
All done!
TrainState.MOVING
25 26
(6, 22) (7, 22)
All done!
TrainState.MOVING
31 32
(2, 15) (5, 16)
All done!
TrainState.MOVING
69 70
(2, 14) (7, 8)
All done!
TrainState.MOVING
99 100
(0, 6) (5, 24)
All done!
TrainState.MOVING
46 47
(10, 2) (9, 7)
Episode didn't finish after 100 timesteps.
All done!
TrainState.MOVING
84 85
(21, 3) (24, 20)
All done!
TrainState.MOVING
60 61
(24, 17) (24, 19)
All done!
TrainState.M

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 1/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:262: UserWarning: [WARNING] Changing to Grid mode to place at least 2 cities.
  warnings.warn("[WARNING] Changing to Grid mode to place at least 2 cities.")


All done!
TrainState.MOVING
45 46
(20, 6) (24, 7)
All done!
TrainState.DONE
46 66
None (9, 19)
Timestep 46, action = foward, total score = 0
All done!
TrainState.DONE
13 20
None (9, 6)
Timestep 13, action = foward, total score = 0
All done!
TrainState.DONE
27 39
None (22, 23)
Timestep 27, action = foward, total score = 0
All done!
TrainState.MOVING
43 44
(10, 22) (20, 17)
All done!
TrainState.MOVING
80 81
(21, 16) (6, 7)
Episode didn't finish after 100 timesteps.
All done!
TrainState.STOPPED
35 36
(10, 5) (5, 24)
All done!
TrainState.MOVING
43 44
(6, 20) (8, 15)
All done!
TrainState.MOVING
55 56
(6, 24) (19, 16)
All done!
TrainState.MOVING
79 80
(23, 22) (18, 10)
All done!
TrainState.MOVING
35 36
(11, 29) (17, 24)


/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 2/4
  warnings.warn(city_warning)


All done!
TrainState.MOVING
59 60
(17, 20) (11, 23)
All done!
TrainState.DONE
18 29
None (9, 6)
Timestep 18, action = foward, total score = 0
All done!
TrainState.MOVING
62 63
(12, 18) (7, 15)
Episode didn't finish after 100 timesteps.
All done!
TrainState.MOVING
89 90
(5, 6) (23, 5)
All done!
TrainState.STOPPED
38 39
(18, 6) (21, 12)
All done!
TrainState.MOVING
90 91
(23, 17) (10, 24)
All done!
TrainState.DONE
72 95
None (23, 10)
Timestep 72, action = foward, total score = 0
Episode didn't finish after 100 timesteps.
All done!
TrainState.STOPPED
42 43
(23, 21) (23, 13)
All done!
TrainState.DONE
44 66
None (9, 18)
Timestep 44, action = foward, total score = 0
All done!
TrainState.DONE
47 71
None (17, 25)
Timestep 47, action = left, total score = 0
All done!
TrainState.MOVING
93 94
(9, 29) (24, 24)
All done!
TrainState.STOPPED
73 74
(17, 22) (8, 22)
All done!
TrainState.STOPPED
72 73
(22, 15) (23, 9)
All done!
TrainState.MOVING
22 23
(6, 6) (6, 10)
All done!
TrainState.MOVING
73 74
(28,